<a href="https://colab.research.google.com/github/kaneelgit/msi_voxceleb/blob/main/Multimodal_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<font size = '6'> <center>**Multimodal Classification**</center></font>

Here I am using a multimodal approach. I will be using both video and audio data for gender classification. The videos feature extraction will be done by a 3D CNN model as before and the audio classification will be done by a 2D classification model as before. 

VoxCeleb dataset - https://www.robots.ox.ac.uk/~vgg/data/voxceleb/

In [1]:
#import libraries
from urllib.request import urlopen
from zipfile import ZipFile

from IPython.display import HTML
from base64 import b64encode
import matplotlib.pyplot as plt

import os
import glob
import numpy as np
import pandas as pd
import cv2     # for capturing videos
import math
import random

import moviepy.editor as mp
from IPython.display import clear_output

import IPython
import librosa

from scipy.io import wavfile
import os

Imageio: 'ffmpeg-linux64-v3.3.1' was not found on your computer; downloading it now.
Try 1. Download from https://github.com/imageio/imageio-binaries/raw/master/ffmpeg/ffmpeg-linux64-v3.3.1 (43.8 MB)
Downloading: 8192/45929032 bytes (0.0%)2334720/45929032 bytes (5.1%)5292032/45929032 bytes (11.5%)8396800/45929032 bytes (18.3%)11673600/45929032 bytes (25.4%)15024128/45929032 bytes (32.7%)18235392/45929032 bytes (39.7%)21389312/45929032 bytes (46.6%)24485888/45929032 bytes (53.3%)27533312/45929032 bytes (59.9%)30760960/45929032 bytes (67.0%)33816576/45929032 bytes (73.6%)37076992/45929032 bytes (80.7%)

# **Download and unzip the data**

In [ ]:
#get url
!wget "https://thor.robots.ox.ac.uk/~vgg/data/voxceleb/vox1a/vox2_test_mp4.zip"

--2021-10-17 19:32:51--  https://thor.robots.ox.ac.uk/~vgg/data/voxceleb/vox1a/vox2_test_mp4.zip
Resolving thor.robots.ox.ac.uk (thor.robots.ox.ac.uk)... 129.67.95.98
Connecting to thor.robots.ox.ac.uk (thor.robots.ox.ac.uk)|129.67.95.98|:443... connected.
HTTP request sent, awaiting response... 

In [ ]:
!unzip vox2_test_mp4.zip

# **Organizing the data**

In the cells below I am collecting all the video paths in a list. I will be then using these videos to extract the audios and their spectrograms to feed in to the neural network.

In [ ]:
#get video paths
vid_paths = []

for path, directories, files in os.walk('/content/mp4/'):

  for file in files:

    vid_paths.append(str(path) + '/' + str(file))

#number of videos available
print('Number of videos available: ', len(vid_paths))

#shuffle video paths. I have used a random seed of 4 to shuffle.
random.seed(4)
random.shuffle(vid_paths)

# **Upload the CSV file and clean meta data**


The csv file with meta data is in my github repository. (https://github.com/kaneelgit/msi_voxceleb). You have to upload the csv file to colab before running the following cells.

In [ ]:
#some functions to clean the csv file
#del spaces from the ids and gender
def del_spaces(string):
  
  string = string.replace(' ', '')

  return string

#upload csv file from github before running
#open csv file
df = pd.read_csv('/content/vox2_meta.csv')
df.head(5)

#clean the dataset

#apply the function to get rid of spaces
df['VoxCeleb2 ID'] = df['VoxCeleb2 ID'].apply(del_spaces)
df['VGGFace2 ID'] = df['VGGFace2 ID'].apply(del_spaces)
df['Gender'] = df['Gender'].apply(del_spaces)
df['Set'] = df['Set'].apply(del_spaces)


#get the test data
df2 = df[df['Set'] == 'test']
df2['Gender'] = df2['Gender'].astype('category')

df2.head(5)

# **Create a Labels List**

Here I am going through the ids of all the video files and picking the gender from the CSV file. Note that there are double the size of male videos compared to female. So I will be taking every other video file to balance the dataset.

In [ ]:
#iterate through the vid_paths get the id of the person and get if the person is female or male. If the person is male its a 1 and female its a 0
labels = []

#get only half of the male videos.
count = 0
video_files = []

#iterate
for path in vid_paths:
  
  #get id number
  id_str =  path[13:20]

  #get if the subject is male or female from the csv
  gender = df2.loc[df2['VoxCeleb2 ID'] == str(id_str)]['Gender'].values[0]

  if gender == 'm':
    if count % 2 == 0:
      labels.append(1)
      video_files.append(path)
    count += 1
  
  else:
    labels.append(0)
    video_files.append(path)

# **Create a data generator**

Here I am creating a data generator to feed audio and video data to the model. 

In [ ]:
#functions to extract the spectrogram and video to array

def vid_to_array(path):
  
  r = np.zeros([224, 224, 15])
  g = np.zeros([224, 224, 15])
  b = np.zeros([224, 224, 15])

  #start count
  count = 0
  idx = 0

  #video capture
  cap = cv2.VideoCapture(path)

  while(cap.isOpened()):
      #frameId = cap.get(1) #current frame number
      ret, frame = cap.read()

      if (count + 1) % 5 == 0:
        r[:, :, idx] = frame[:, :, 0]/255
        g[:, :, idx] = frame[:, :, 1]/255
        b[:, :, idx] = frame[:, :, 2]/255

      if count == 74:
        break
      count += 1
  cap.release()

  return np.stack([r, g, b], axis = 3)

#convert from audio to a mel diagram
def aud_to_mel(path):
  
  #get video from path
  video = mp.VideoFileClip(path)
  video = video.subclip(0, 3)

  #get the audio
  x = video.audio.to_soundarray()

  #get audio track to array
  #x = aud.to_soundarray()

  #sampling rate
  fs = 44100
  
  #channel 1
  ch1 = librosa.feature.melspectrogram(x[:, 0], sr = fs)
  mel_ch1 = librosa.amplitude_to_db(ch1, ref = np.min)

  #channel 2
  # ch2 = librosa.feature.melspectrogram(x[:, 1], sr = fs)
  # mel_ch2 = librosa.amplitude_to_db(ch2, ref = np.min)
  
  return mel_ch1.reshape(mel_ch1.shape[0], mel_ch1.shape[1], 1)

In [ ]:
#create a data generator
def data_generator(paths, labels, batch_size = 16):

  while True:

    #get the number of batches
    number_of_batches = len(paths) // batch_size

    #set up video batches and label batches
    batches = [paths[i : i + batch_size] for i in range(0, len(paths), batch_size)]
    label_batches = [labels[i : i + batch_size] for i in range(0, len(paths), batch_size)]

    for b in range(number_of_batches):
      
      #create array to store video batch
      x_batch_video = np.zeros([batch_size, 224, 224, 15, 3])
      
      #create array to store audio batch
      x_batch_audio = np.zeros([batch_size, 128, 259, 1])
      
      #labels
      y_batch = np.zeros([batch_size, 1])

      #get video batch and label batch
      batch = batches[b]
      label_batch = label_batches[b]

      #iterate through the batch path and get the array 
      for i, p in enumerate(batch):
        
        #get video array and store the result
        vid_array = vid_to_array(p)
        x_batch_video[i, :] = vid_array

        #get audio array and store the result
        audio_array = aud_to_mel(p)
        x_batch_audio[i, :] = audio_array

        #y batch
        y_batch[i, :] = label_batch[i]
    
      yield {'video': x_batch_video, 'audio': x_batch_audio}, {'class': y_batch}


# **Train/validation split**

Here I have manually seperated the train and validation set because I ran into memory issues in colab.

In [ ]:
#split to train and validation set
train_idx = int(np.floor(len(labels) * 0.9))

#split the train and test data accordingly
train_paths = video_files[:train_idx]
train_labels = labels[:train_idx]

#split the test data
val_paths = video_files[train_idx:]
val_labels = labels[train_idx:]

In [ ]:
#validation set
validation_set_vid = np.zeros([150, 224, 224, 15, 3])
validation_set_aud = np.zeros([150, 128, 259, 1])
validation_labels = np.zeros([150, 1])

#select 100 random indexes from the validation set
#rand_val_idx = np.random.choice(np.arange(len(val_paths)), 150)
rand_val_idx = np.arange(len(val_paths[:150]))

for i, idx in enumerate(rand_val_idx):

  #vid path
  path = val_paths[idx]
  label = val_labels[idx]

  validation_set_vid[i, :] = vid_to_array(path)
  validation_set_aud[i, :] = aud_to_mel(path)
  validation_labels[i, :] = label

In [ ]:
#create a train set
training_set_vid = np.zeros([800, 224, 224, 15, 3])
training_set_aud = np.zeros([800, 128, 259, 1])
training_labels = np.zeros([800, 1])

#select 1000 files from the train files
rand_train_idx = np.arange(len(train_paths[:800]))

for i, idx in enumerate(rand_train_idx):

  #aud path
  path = train_paths[idx]
  label = train_labels[idx]

  training_set_vid[i, :] = vid_to_array(path)
  training_set_aud[i, :] = aud_to_mel(path)
  training_labels[i, :] = label

# **Multimodal Architecture**

Here I create a multimodal architecture to use both audio and video data for classification


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
import numpy as np
import scipy.misc
from tensorflow.keras.applications.resnet_v2 import ResNet50V2
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet_v2 import preprocess_input, decode_predictions
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Add, Dense, Average, Activation, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D, Dropout, Concatenate
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.initializers import random_uniform, glorot_uniform, constant, identity
from tensorflow.python.framework.ops import EagerTensor
from tensorflow.python.keras.utils.vis_utils import plot_model

In [ ]:
#3d cnn for video
def cnn_3dmodel(x):
    
    inputs = x
    x = tf.keras.layers.Conv3D(16, kernel_size = (3, 3, 1),activation = 'relu')(inputs)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.MaxPool3D(pool_size = (2, 2, 2))(x)
    
    x = tf.keras.layers.Conv3D(32, kernel_size = (3, 3, 1), activation = 'relu')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.MaxPool3D(pool_size = (2, 2, 2))(x)
    
    x = tf.keras.layers.Conv3D(64, kernel_size = (3, 3, 1), activation = 'relu')(x) 
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.MaxPool3D(pool_size = (2, 2, 1))(x)

    return x

#video inp
input_video_tmp = np.zeros([16, 224, 224, 15, 3])

#creat video layers
input_vid = Input(input_video_tmp[0, :].shape, name = 'video')
out_vid = cnn_3dmodel(input_vid)
out_vid = Flatten()(out_vid)
out_vid = Dropout(0.5)(out_vid)
out_vid = Dense(1024, activation = 'relu')(out_vid)
out_vid = Dense(128, activation = 'relu')(out_vid)
output_video = Dense(1, activation = 'sigmoid', name = 'video_class')(out_vid)


In [ ]:
#2d cnn for audio
def cnn_2dmodel(x):
    
    inputs = x
    x = tf.keras.layers.Conv2D(16, kernel_size = (3, 3),activation = 'relu')(inputs)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.MaxPool2D(pool_size = (2, 2))(x)
    
    x = tf.keras.layers.Conv2D(32, kernel_size = (3, 3), activation = 'relu')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.MaxPool2D(pool_size = (2, 2))(x)
    
    x = tf.keras.layers.Conv2D(64, kernel_size = (3, 3), activation = 'relu')(x) 
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.MaxPool2D(pool_size = (2, 2))(x)

    return x

#create audio layers
input_audio_tmp = np.zeros([16, 128, 259, 1])
input_aud = Input(input_audio_tmp[0, :].shape, name = 'audio')
out_aud = cnn_2dmodel(input_aud)
out_aud = Flatten()(out_aud)
out_aud = Dropout(0.5)(out_aud)
out_aud = Dense(1024, activation = 'relu')(out_aud)
out_aud = Dropout(0.8)(out_aud)
out_aud = Dense(128, activation = 'relu')(out_aud)
output_audio = Dense(1, activation = 'sigmoid', name = 'audio_class')(out_aud)


In [ ]:
#create the combined model
combined_out = tf.keras.layers.Concatenate(axis=1)([output_video, output_audio])
combined_out = Dense(1, activation = 'sigmoid', name = 'class')(combined_out)
combined_model = Model([input_vid, input_aud], combined_out)

#plot the combined model
plot_model(combined_model, show_shapes = True, show_layer_names = True, to_file = 'test_model.png')

In [ ]:
#compile the model
opt = tf.optimizers.Adam(0.001)

#learning rate scheduler
def lr_schedule(epoch, lr):
    
    if (epoch + 1) % 5 == 0:
        
        lr *= 0.2
    
    return max(lr, 3e-7)

opt = tf.keras.optimizers.SGD(
    learning_rate=0.01, momentum=0.0, nesterov=False, name='SGD')

#compile model
combined_model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = 'accuracy')

#callbacks
#create a callback to stop the model when loss hit plateur
cb = tf.keras.callbacks.EarlyStopping(monitor = 'loss', patience = 5, mode = 'auto')

cb_lr = tf.keras.callbacks.LearningRateScheduler(lr_schedule)

#fit model
history = combined_model.fit(data_generator(train_paths, train_labels, batch_size = 8), validation_data = ([validation_set_vid, validation_set_aud], validation_labels),\
                    steps_per_epoch = 100, epochs = 10, callbacks = [cb, cb_lr])